# **Candidate Numbers: 2301435, 2300151, 2307521**

**Import the desired packages**

In [ ]:
!pip install bs4
!pip install requests
!pip install yfinance
import math
import pandas as pd
import numpy as np
import datetime as dt
import requests
import json
import seaborn as sns
import yfinance as yf
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

# **Data Collection**



**Get ticker Data from yahoo finance and define it as a function**

In [ ]:
def ticker_data(ticker, start_date, end_date):
    # Use the yf.Ticker() function to get the historical data for the specified ticker
    ticker_data = yf.Ticker(ticker)
    #Use the .history() method to get the historical data for the specified date range
    ticker_data_history = ticker_data.history(start=start_date, end=end_date)
    #Reset the index to convert the date index to a column
    ticker_data_history = ticker_data_history.reset_index()
    #Rename the "index" column to "Date"
    ticker_data_history = ticker_data_history.rename(columns={"index": "Date"})
    #Convert "Date" column to datetime type
    ticker_data_history['Date'] = pd.to_datetime(ticker_data_history['Date']).dt.date

    #Adds the ticker symbol to each column name in the dataframe
    for col_name in ticker_data_history.columns[1:]:
        new_col_name = col_name + "_" + ticker
        ticker_data_history = ticker_data_history.rename(columns={col_name: new_col_name})

    #Create the data frame
    ticker_data_clean = ticker_data_history[["Date", "Open_" + ticker, "High_" + ticker, "Low_" + ticker, "Close_" + ticker]]
    df_ticker = pd.DataFrame(data=ticker_data_clean)

    # Create the spread variable we want
    spread = "Spread_" + ticker
    df_ticker[spread] = df_ticker["High_" + ticker] - df_ticker["Low_" + ticker]

    #Create log spread variable
    log_spread = "Log_Spread_" + ticker
    df_ticker[log_spread] = np.log(df_ticker[spread])

    # Return daily % change in spread
    df_ticker.fillna(value=0, inplace=True)
    spread_pct_change = "Spread_pct_change_" + ticker
    df_ticker[spread_pct_change] = df_ticker[spread].pct_change()

    #filter for business days to keep consisten with forex days
    df_ticker = df_ticker.set_index('Date').asfreq('B').reset_index()

    #Set any na or infinite values to 0
    df_ticker.replace([np.inf, -np.inf], 0, inplace=True)
    df_ticker.fillna(value=0, inplace=True)



    return df_ticker


**Get the desired cyrpto and forex pairings**
We shall use Bitcoin as the crypto since it is often cited the most and has the greatest financial instruments with BTC as the underlying asset, as well as using the top 5 forex pairings which account for roughly 55% of the total forex trading volume.

In [ ]:
df_btc = ticker_data('BTC-USD','2015-01-01','2023-01-01')
#top 5 forex pairings (in order)
df_EUR_USD = ticker_data('EURUSD=X','2015-01-01','2023-01-01')
df_USD_GBP = ticker_data('USDGBP=X','2015-01-01','2023-01-01')
df_USD_JPY = ticker_data('USDJPY=X','2015-01-01','2023-01-01')
df_AUD_USD = ticker_data('AUDUSD=X','2015-01-01','2023-01-01')
df_USD_CAD = ticker_data('USDCAD=X','2015-01-01','2023-01-01')

#merge dataframes
from functools import reduce
df_list = [df_btc, df_EUR_USD, df_USD_GBP, df_USD_JPY, df_AUD_USD, df_USD_CAD]
df_merged = reduce(lambda left,right: pd.merge(left,right,on='Date', how='inner'), df_list)

**Get data for the further explanatory variables defined in our write-up**

In [ ]:
###UPLOAD the CSV file provided in the Zip file to access the data
from google.colab import files
uploaded = files.upload()
import io
df_btc_elec = pd.read_csv(io.BytesIO(uploaded['BECI.csv']),header=1)
import io
df_btc_elec = pd.read_csv(io.BytesIO(uploaded['BECI.csv']),header=1)
df_btc_elec = df_btc_elec.rename(columns={'Date and Time': 'Date'})

# convert "Date" column to datetime format
df_btc_elec['Date'] = pd.to_datetime(df_btc_elec['Date'])

# set start and end dates
start_date = '2015-01-01'
end_date = '2023-01-01'

#filter the bitcoin electrcity DataFrame based on the start and end dates
df_btc_elec = df_btc_elec[(df_btc_elec['Date'] >= start_date) & (df_btc_elec['Date'] < end_date)]

#filter for business days to keep consistent with the other variable days
df_btc_elec = df_btc_elec.set_index('Date').asfreq('B').reset_index()

#additional variables for bond and market index per defined
df_spy = ticker_data('SPY', '2015-01-01', '2023-01-01')
df_fvx = ticker_data('^FVX', '2015-01-01', '2023-01-01')

#combine btc electricity, bond and market index variables
df_var_list = [df_btc,df_btc_elec,df_spy,df_fvx]
df_var_list = reduce(lambda left,right: pd.merge(left,right,on='Date', how='inner'), df_var_list)

Saving BECI.csv to BECI.csv


/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# **Data Visualisation**

**Create plot for the forex pair spread**

In [ ]:
# Create a figure and axis object
sns.set_theme(style="ticks")
fig, ax = plt.subplots(figsize=(12,6))

# Plot the Spread column for each forex pairing
sns.lineplot(data=df_merged, x="Date", y="Spread_EURUSD=X", label="EUR/USD", linewidth=0.8)
sns.lineplot(data=df_merged, x="Date", y="Spread_USDGBP=X", label="USD/GBP", linewidth=0.8)
sns.lineplot(data=df_merged, x="Date", y="Spread_AUDUSD=X", label="AUD/USD", linewidth=0.8)
sns.lineplot(data=df_merged, x="Date", y="Spread_USDCAD=X", label="USD/CAD", linewidth=0.8)

#create 2nd axis for USDJPY since its too large
ax2 = ax.twinx()
sns.lineplot(data=df_merged, x="Date", y="Spread_USDJPY=X", label="USD/JPY", linewidth=0.8,color='Purple')

# Set the title and axis labels
ax.set_title("Spread of Forex Pairings")
ax.set_xlabel("Date")
ax.set_ylabel("Spread")

# Set the legend
ax.legend()
ax2.legend(loc='upper center')

# Set the title and axis labels
ax2.set_title("Spread of Forex Pairings")
ax2.set_xlabel("Date")
ax2.set_ylabel("Spread for USD/JPY")


# Show the plot
plt.show()

**Plot the forex and btc spreads in 1 figure**

In [ ]:
# Create a figure and axis object
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12,6))

# Plot the Spread column for each forex pairing
sns.lineplot(data=df_merged, x="Date", y="Spread_EURUSD=X", label="EUR/USD", linewidth=0.8, ax=axes[0,0], color='red')
sns.lineplot(data=df_merged, x="Date", y="Spread_USDGBP=X", label="USD/GBP", linewidth=0.8, ax=axes[0,1],color='blue')
sns.lineplot(data=df_merged, x="Date", y="Spread_AUDUSD=X", label="AUD/USD", linewidth=0.8, ax=axes[1,0],color='green')
sns.lineplot(data=df_merged, x="Date", y="Spread_USDCAD=X", label="USD/CAD", linewidth=0.8, ax=axes[1,1],color='orange')
sns.lineplot(data=df_merged, x="Date", y="Spread_USDJPY=X", label="USD/JPY", linewidth=0.8, ax=axes[2,0], color='Purple')
sns.lineplot(data=df_merged, x="Date", y="Spread_BTC-USD", label="BTC", linewidth=0.8, ax=axes[2,1], color='turquoise')

# Set the title and axis labels
for ax in axes.flat:
    ax.set_title("Spread")
    ax.set_xlabel("Date")
    ax.set_ylabel("Spread")

#clean up the spacings
plt.subplots_adjust(wspace=0.5, hspace=1)
# Show the plot
plt.show()

**Show % change in spread**

In [ ]:
#Create a figure and axis object
fig, ax = plt.subplots(figsize=(12,6))

#Set the y limit since there are some outliers
ax.set_ylim([0, 10])

#Plot the Spread column for each forex pairing
sns.lineplot(data=df_merged, x="Date", y="Spread_pct_change_EURUSD=X", label="EUR/USD", linewidth=0.8)
sns.lineplot(data=df_merged, x="Date", y="Spread_pct_change_USDGBP=X", label="USD/GBP", linewidth=0.8)
sns.lineplot(data=df_merged, x="Date", y="Spread_pct_change_AUDUSD=X", label="AUD/USD", linewidth=0.8)
sns.lineplot(data=df_merged, x="Date", y="Spread_pct_change_USDCAD=X", label="USD/CAD", linewidth=0.8)
sns.lineplot(data=df_merged, x="Date", y="Spread_pct_change_USDJPY=X", label="USD/JPY", linewidth=0.8)

#Set the title and axis labels
ax.set_title("Spread % change in Forex Pairings")
ax.set_xlabel("Date")
ax.set_ylabel("Spread % change")

#Set the legend
ax.legend()

#Show the plot
plt.show()

**Show for bitcoin computing electricity consumption**

In [ ]:
#Create a figure and axis object
fig, ax = plt.subplots(figsize=(12,6))

#Plot the Spread column for each forex pairing
sns.lineplot(data=df_var_list, x="Date", y="Spread_BTC-USD", label="BTC Spread", linewidth=0.8, color='pink')

#Set the title and axis labels
ax.set_title("Spread change for BTC and BTC Electrcity consumption")
ax.set_xlabel("Date")
ax.set_ylabel("BTC Spread change")

#create 2nd axis for USDJPY since its too large
ax2 = ax.twinx()
sns.lineplot(data=df_var_list, x="Date", y='annualised consumption GUESS, TWh', label="Annualised Electricity Consumption", linewidth=0.8)

#Set the legend
ax.legend()
ax2.legend(loc='upper left')

#Show the plot
plt.show()

**ScatterPlot showing btc electrcity consumption and bitcoin spread**

In [ ]:
# Create a linear regression plot
sns.lmplot(x='annualised consumption GUESS, TWh', y='Spread_BTC-USD', data=df_var_list, height=6, aspect=2,line_kws={'color': 'orange'})

# Set the title of the plot
plt.title('Scatter Plot of BTC electricity consumption vs BTC spread')

# Show the plot
plt.show()


# **Data Analysis**

**Multi-linear model with the forex pairs**

In [ ]:
# define the dependent variable (y) and independent variable (x)
y = df_btc['Log_Spread_BTC-USD']

#multi-linear variable with all 5
x = df_merged[['Log_Spread_EURUSD=X','Log_Spread_USDGBP=X','Log_Spread_AUDUSD=X','Log_Spread_USDCAD=X','Log_Spread_USDJPY=X']]

# create the multi-linear regression model
multi_model = sm.OLS(y, x)

# print the summary of the multi-model
multi_model.fit().summary()

**Multi-linear model with the other explanatory variables**

In [ ]:
# define the dependent variable (y) and independent variable (x)
y = df_var_list['Log_Spread_BTC-USD']

#multi-linear variable with all 5
x = df_var_list[['annualised consumption GUESS, TWh','Log_Spread_SPY','Log_Spread_^FVX']]

# create the multi-linear regression model
multi_model = sm.OLS(y, x)

# print the summary of the multi-model
multi_model.fit().summary()